# Project
## 3rd Notebook : Elasticsearch
##### Students:
* Lilia IZRI      (DS)
* Yacine MOKHTARI (DS)
* Alexandre COMBEAU (DS)

##### Report
[PENSER A METTRE UN LIEN ICI]


### A quick test to see if the ... etc 

In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch("http://host.docker.internal:9200") # Same port when running elasticsearch with docker

In [3]:
es.get(index="test", id=1)

/tmp/ipykernel_27661/1768325765.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.get(index="test", id=1)


ObjectApiResponse({'_index': 'test', '_type': '_doc', '_id': '1', '_version': 608, '_seq_no': 608, '_primary_term': 1, 'found': True, '_source': {'user': ' 1TRUE_AMERICA', 'tweet': " @Jim222001 I do though acknowledge the possibility of someone lying in testimony, under oath. So just because it's… https://t.co/AdhOlJDAJu", 'date': ' Fri May 06 02:21:30 +0000 2022', 'lat': ' 36.2231079', 'lon': ' -121.3877428', 'hashtags': ' ', 'sentiment': ' 0', 'id': ' 992117141477339136'}})

In [5]:
es.get(index="test", id=1)

/tmp/ipykernel_27661/1768325765.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.get(index="test", id=1)


ObjectApiResponse({'_index': 'test', '_type': '_doc', '_id': '1', '_version': 618, '_seq_no': 618, '_primary_term': 1, 'found': True, '_source': {'user': ' CityReport', 'tweet': ' "Fühlte mich so machtlos": Heard: Depp missbrauchte mich mit Flaschen https://t.co/bYlcYTAHy7 https://t.co/J8q7ZI5vzc', 'date': ' Fri May 06 02:21:32 +0000 2022', 'lat': ' 52.4009309', 'lon': ' 13.0591397', 'hashtags': ' ', 'sentiment': ' 0', 'id': ' 718145940'}})